In [ ]:
# Interactive Movie recommendation system
# typein movie name and get recommendations 
# step 1 - to dowload the data for the movie ratings - read in the data 
# step 2 build a seach engine to search throguh the movies , cleaned the titles and built a movie search engine  
# step 3 then build a system to give the recommendations - read in ratings and created a function to recommend movies based on  a movie watcehd before - building a movie recommendation  

In [3]:
import pandas as pd 

In [4]:
movies = pd.read_csv("movies.csv")

In [86]:
movies.head()

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji (1995)
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II (1995)


In [87]:
# to build  a search engine , use regex to clean the titles 
import re 

# define function to clean the title like parenthesis and extra characters 
# this function looks for characters that arent a digit space or letter, so it remvos parenthese dasheds 
# it looks for characters that arent a digit,  space or letter and will remove them 

def clean_title(title): 
    re.sub("[^a-zA-Z0-9]", "", title)
    return title

In [88]:
# adding a new column and calling the clean title function 
movies["clean_title"] = movies["title"].apply(clean_title)

In [89]:
movies 

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji (1995)
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II (1995)
...,...,...,...,...
62418,209157,We (2018),Drama,We (2018)
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul (2001)
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems (2018)
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing (2001)


In [90]:
# building a search engine by creating a tfidf matrix
# need to convert the titles into numbers / binary numbers by word - creating term frequency
# inverse document frequency- helps search engine to find unique terms, we get a vector that describes the movie title   

In [104]:

from sklearn.feature_extraction.text import TfidfVectorizer

# initialise the class , to look at engrams not just one word but two consecutive words 

vectorizer = TfidfVectorizer(ngram_range=(1,2))

# use vectorizer to turn set of titles into a matrix 

tfidf = vectorizer.fit_transform(movies["clean_title"])

In [133]:
# to compute the similarity with the different films 

from sklearn.metrics.pairwise import cosine_similarity 

import numpy as np 

def search(title): 
# clean the titles entered, and vectorizer used to turn set of terms into a set of numbers 
    title = clean_title(title) 
    query_vec = vectorizer.transform([title])
# to find the similarity in the search term  with the title entered in the search engine, compares query term to each title 
    similarity = cosine_similarity(query_vec,tfidf).flatten()

# to find the five titles with the greatest similarity to the search term 
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices][::-1]
# to reverse the order of the matrix [::-1]
    return results 

In [134]:
results 

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)
14813,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 (2010)
59767,201588,Toy Story 4 (2019),Adventure|Animation|Children|Comedy,Toy Story 4 (2019)
3021,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 (1999)
20497,106022,Toy Story of Terror (2013),Animation|Children|Comedy,Toy Story of Terror (2013)


In [135]:
# in order to insert the search engine function 

In [136]:
import ipywidgets as widgets 
from IPython.display import display

# to create a text input widget , with default value toystory (this input is a dictionary)
movie_input = widgets.Text(
    value="Toy Story",
    description="Movie Title:",
    disabled=False
)

movie_list = widgets.Output()
#to create an output widget 
# input should be 
def on_type(data):
    with movie_list: 
        movie_list.clear_output()
        # grabbing the input from the dictionary 
        title = data["new"]
        if len(title) >5:
            # this searches the set of titles for the title , then it displays it in the output widget- movielist  
            display(search(title))
# to connect the input widget tot he ontype function - so the ontype method is called when anything is done with movieinout
# hwne event vlaue is called it will run the ontype function 
movie_input.observe(on_type, names='value')

display(movie_input, movie_list)



Text(value='Toy Story', description='Movie Title:')

Output()

In [ ]:
# reading in the movies data - building the recommendation system 

In [138]:
# reading in the ratings file 
ratings = pd.read_csv("ratings.csv")


In [139]:
ratings 

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [143]:
ratings.dtypes 
# to find the datatypes of the data 

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [144]:
movie_id = 1 

In [149]:
# to find the users with similar movie interests - defined as people who liked the movie and those that rated it high, those that watched the movie searched and anyone who gave it a 5/5 
similar_users = ratings[(ratings["movieId"]==movie_id) & (ratings["rating"] > 4)]["userId"].unique()                   

In [150]:
# this calls a set of an array of the users 
similar_users

array([    36,     75,     86, ..., 162527, 162530, 162533])

In [155]:
# now to find the similar films that the users in the array liked 

similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

In [156]:
similar_user_recs

5101            1
5105           34
5111          110
5114          150
5127          260
            ...  
24998854    60069
24998861    67997
24998876    78499
24998884    81591
24998888    88129
Name: movieId, Length: 1358326, dtype: int64

In [158]:
# to get the movie id from the ratings and in movies data set 
# to find the movies that only that greater than 10 % of users similar to us liked 

In [171]:
# to count how much a particular movie appears in the dataset , dividing by length of users to get a percentage 

similar_user_recs = similar_user_recs.value_counts()/ len(similar_users)

# to only take the ones >10%
similar_user_recs = similar_user_recs[similar_user_recs > .1]

similar_user_recs 

1        1.000000
318      0.445607
260      0.403770
356      0.370215
296      0.367295
           ...   
953      0.103053
551      0.101195
1222     0.100876
745      0.100345
48780    0.100186
Name: movieId, Length: 113, dtype: float64

In [172]:
# finding how much all the users like movies, so finding movies that users liked that are similar
#all the users who have watched all of the ratings for the movies in the set - shows all the users who have watched the movies that were recommended to us 
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]


In [173]:
# to find what percentage of all users recommend these movies , so a dataframe will be created 
# is better to have a bigger differential between movies recommended to differentiate between movies that everyone likes or movies rated highly that are recommended that are similar to what has been put in the search engine
all_users_recs = all_users["movieId"].value_counts()/ len(all_users["userId"].unique())




In [174]:
all_users_recs

318      0.342220
296      0.284674
2571     0.244033
356      0.235266
593      0.225909
           ...   
551      0.040918
50872    0.039111
745      0.037031
78499    0.035131
2355     0.025091
Name: movieId, Length: 113, dtype: float64

In [185]:
#to compare the percentages - use pandas  to create a dataframe and concatenation to combine two series 
rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis=1)
rec_percentages.columns = ["similar", "all"]

In [186]:
rec_percentages 
# gives each of the movies that were recommended and how much  users who are similar to the me like them and how much everyone ekse likes them - so we need a big differential ebtween the similar and all column 

,similar,all
1,1.000000,0.124728
318,0.445607,0.342220
260,0.403770,0.222207
356,0.370215,0.235266
296,0.367295,0.284674
...,...,...
953,0.103053,0.045792
551,0.101195,0.040918
1222,0.100876,0.066877
745,0.100345,0.037031


In [187]:
# to find the ratio between the percentages 
rec_percentages["score"] = rec_percentages["similar"]/rec_percentages["all"]

In [188]:
# to rearrage the order using pandas sort value method to sort values 
rec_percentages = rec_percentages.sort_values("score", ascending =False)

In [189]:
# the higher the score the better the recommendation
rec_percentages

,similar,all,score
1,1.000000,0.124728,8.017414
3114,0.280648,0.053706,5.225654
2355,0.110539,0.025091,4.405452
78499,0.152960,0.035131,4.354038
4886,0.235147,0.070811,3.320783
...,...,...,...
2858,0.216724,0.167634,1.292845
296,0.367295,0.284674,1.290232
79132,0.166817,0.131384,1.269693
4973,0.142501,0.112405,1.267747


In [197]:
# take the top ten recommendations and merge it with the movie data,lefindex true means use that index left column for movieid, then merge it on right with movies data
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
0,1.000000,0.124728,8.017414,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)
3021,0.280648,0.053706,5.225654,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 (1999)
2264,0.110539,0.025091,4.405452,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,"Bug's Life, A (1998)"
14813,0.152960,0.035131,4.354038,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 (2010)
4780,0.235147,0.070811,3.320783,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,"Monsters, Inc. (2001)"
580,0.216618,0.067513,3.208539,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,Aladdin (1992)
6258,0.228139,0.072268,3.156862,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,Finding Nemo (2003)
587,0.179400,0.059977,2.991150,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,Beauty and the Beast (1991)
8246,0.203504,0.068453,2.972889,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,"Incredibles, The (2004)"
359,0.253411,0.085764,2.954762,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,"Lion King, The (1994)"


In [198]:
# building the recommendation function 

In [205]:
def find_similar_movies(movie_id):
    
    #finding recommendations from users similar 
    similar_users = ratings[(ratings["movieId"]==movie_id) & (ratings["rating"] > 4)]["userId"].unique()   
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    # adjusting to only have recommendations that 10% of users have recommended
    similar_user_recs = similar_user_recs.value_counts()/ len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .1]
    
    # how common the recommendations were amongst the users 
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_users_recs = all_users["movieId"].value_counts()/ len(all_users["userId"].unique())
    
    # creating a score 
    rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    rec_percentages["score"] = rec_percentages["similar"]/rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending =False)
    

    # and returning the score and the essential columns from the data table
    
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [207]:
# now building the widget that does this automatically 
# creating the input 
movie_name_input= widgets.Text(
 value= "Toy Story", 
    description= "Movie Title:",
    disabled=False 
)

#creating output widget 
recommendation_list = widgets.Output()

# now creating the ontype function 

def on_type(data): 
    with recommendation_list: 
        recommendation_list. clear_output()
        title = data["new"]
        if len (title)>5: 
            results = search(title)
            movie_id = results.iloc[0]["movieId"] # result that had the highest confidence and find similar movies to the movie in the top row 
            display(find_similar_movies(movie_id))
 
movie_name_input.observe(on_type, names="value")

# display the widget 
display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [ ]:
# to improve can build a search engine based on film genres , or use film genres to improve the exisiting 